<a href="https://colab.research.google.com/github/Ansi4Ansi/Google_colab/blob/main/backup_02_Model_fine_tuning_with_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Файнтюнинг Transformer с помощью Pytorch

На этом семинаре мы разберем как обучить модель на pytorch и какие есть нюансы:
- вспомним зачем в pytorch нужны объекты dataset и dataloader и какие есть best practices по работе с ними
- вспомним как работает автоматическое дифференцирование в pytorch
- Обучим модель классификации с помощью оптимизированной distilbert и увидим что маленькие модели тоже могут работать хорошо


In [ ]:
!pip install transformers==4.24.0
!pip install -U sentence-transformers==2.2.2
!pip install datasets==2.14.4
!pip install sentencepiece==0.1.99

In [ ]:
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification, PreTrainedTokenizer, PreTrainedTokenizerFast
import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader, Dataset
import numpy as np
import typing as tp

## 2. Данные

Возьмем датасет imdb с хаба Hugging Face
https://huggingface.co/datasets/imdb

Данные пресдтавляют собой коллекцию отзывов на фильмы и разметку сентимента (позитивный или негативный)

In [ ]:
imdb = load_dataset("imdb", split="test")

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Приведем данные к формату pandas

In [ ]:
imdb_df = imdb.to_pandas()

In [ ]:
imdb_df.head()

,text,label
0,I love sci-fi and am willing to put up with a ...,0
1,"Worth the entertainment value of a rental, esp...",0
2,its a totally average film with a few semi-alr...,0
3,STAR RATING: ***** Saturday Night **** Friday ...,0
4,"First off let me say, If you haven't enjoyed a...",0


Возьмем 1000 случайных строк для тестирования модели

In [ ]:
imdb_sample = imdb_df.sample(n=1000, random_state=2023)

Посмотрим на распределение классов

In [ ]:
imdb_sample["label"].value_counts()

1    529
0    471
Name: label, dtype: int64

## 3. Протестируем кач-во предобученной модели без дообучения.

Перед тем как мы перейдем к обучению модели на наших данных, давайте посмотрим как готовые модели справляются с данной задачей. Анализ тональности - популярная задача и существует ряд моделей, которые были предобучены под нее.

Мы воспользуемся ```sentiment-roberta-large-english``` - это довольно большая модель и она хороший кандидат для задачи.

In [ ]:
model_name = "siebert/sentiment-roberta-large-english"

model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
config = transformers.AutoConfig.from_pretrained(model_name)

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

### 3.1  Dataset и Dataloader

Как для обучения, так и для применения нам потребуется итерироваться по нашему датасету.
Итерироваться по каждому объекту по отдельности очень не эффективно. Чтобы это исправить в  Pytorch есть абстракции Dataset и Dataloader, которые позволяют эффективно обрабатывать батчи данных.

Читаь про то что такое [Dataset](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html).

Объекты Dataset бывает [map style](https://pytorch.org/docs/stable/data.html#map-style-datasets) и [iterable style](https://pytorch.org/docs/stable/data.html#map-style-datasets).


Чаще всего используют map style dataset
Чтобы его создать, нужно реализовать 2 метода:

In [ ]:
class ExampleDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        encoding = self.tokenizer(text, truncation=True, padding='max_length', max_length=self.max_length, return_tensors='pt')

        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'label': torch.tensor(label, dtype=torch.long)
        }

Хорошая практика, чтобы метод ```__getitem__``` возвращал словарь, а не кортеж значений (в котором по определению отсутствуют ключи и что может привести к путанице).

Создадим экземпляр класса ExampleDataset. Передаем данные, разметку, объект токенизатора и параметр для контроля максимальной длины последовательности.

In [ ]:
example_dataset = ExampleDataset(
    texts=imdb_sample["text"].tolist(),
    labels=imdb_sample["label"].tolist(),
    tokenizer=tokenizer,
    max_length=512
    )

Убедимся что метод ```__getitem__``` работает как ожидается. Напрямую применять нам его не потребуется, но он позже будет вызываться в объекте Dataloader.



In [ ]:
example_dataset[0]

Dataloader принимает на вход объект Dataset и набор опциональных дополнительных параметров и позволяет эффективно итерироваться по батчам данных.

In [ ]:
dataloader = DataLoader(example_dataset, batch_size=16, shuffle=False)

### Полезные фичи Dataloader:

* Возможность загружать данные параллельно, используя несколько CPU (multi processing).
* Возможность делать трансформации данных на лету (например, аугментации, нормализации)
* Возможность использовать разные samplers которые контролируют какие объекты попадают в батч (самые распространенные - Sequential и Random, но можно задать свой sampler)
* Возможность обрабатывать батч на лету (например, привести все объекты батча к одной длине), логика обработки задается функцией collate_fn, которую нужно реализовать самому и передать как аргумент в Dataloader. Этот параметр опциональный. Если его не передать, то будет использован дефолтный collate_fn - почитать как он работает можно [тут](https://pytorch.org/docs/stable/data.html#working-with-collate-fn). Подробнее о рекомендациях как использовать collate_fn [тут](https://discuss.pytorch.org/t/how-to-use-collate-fn/27181).

Под капотом в Dataloader происходит примерно следующее:

```
for indices in batch_sampler:
    yield collate_fn([dataset[i] for i in indices])
```

### Пример iterable dataset и когда это полезно

[WebDataset](https://github.com/webdataset/webdataset) - это объект Dataset, который умеет читаь данные с хранилища при условии что данные лежат в формате tar. Оформлен в виде отдельной библиотеки, полностью интегрирован с pytorch.

Полезно когда нужно из коробки воспользоваться выгрузкой данных из хранилища сразу в скрипт для обучения модели.

### 3.2 Функция для оценки качества модели на валидационном датасете.

Принимает на вход модели и dataloader. Применяет модель и возвращает предсказания и разметку.

In [ ]:
def validate(model, dataloader, device='cuda'):

    model.eval()
    model.to(device)

    valid_preds, valid_labels = [], []

    for batch in dataloader:


        b_input_ids = batch["input_ids"].to(device)
        b_input_mask = batch["attention_mask"].to(device)
        b_labels = batch["label"].to(device)

        with torch.no_grad():
            logits = model(input_ids=b_input_ids, attention_mask=b_input_mask)

        logits = logits[0].detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        batch_preds = np.argmax(logits, axis=1)
        batch_labels = np.concatenate(label_ids.reshape(-1,1))
        valid_preds.extend(batch_preds)
        valid_labels.extend(batch_labels)

    return valid_labels, valid_preds

In [ ]:
valid_labels, valid_preds = validate(model, dataloader)

Посчитаем метрику [F1](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

f1_score(valid_labels, valid_preds)

0.9584905660377359

## 4. Fine tuning модели.

Ранее мы видели, что предобученная модель способна с хорошим качеством решать данную задачу, но та модель очень большая. Можно ли взять модель меньше, дообучить ее на наших данных и получить сравнимое качество?

Чтобы проверить эту гипотезу возьмем дистилированный BERT.

In [ ]:
model_name = "distilbert-base-uncased"

model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
config = transformers.AutoConfig.from_pretrained(model_name)

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'pre_classifier.weight', 'classifier.

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

### 4.1 Данные для обучения.

Возьмем 5 тысяч строк

In [ ]:
imdb_train = load_dataset("imdb", split="train")

In [ ]:
imdb_train_sample = imdb_train.to_pandas().sample(n=5000, random_state=2023)

In [ ]:
imdb_train_sample.label.value_counts(dropna=False)

1    2552
0    2448
Name: label, dtype: int64

Создадим train_dataset и dataloader

In [ ]:
train_dataset = ExampleDataset(
    texts=imdb_train_sample["text"].tolist(),
    labels=imdb_train_sample["label"].tolist(),
    tokenizer=tokenizer,
    max_length=512
    )

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=False)

### 4.2 Функция чтобы обучить модель

In [ ]:
def train(model, train_loader, num_epochs=2, learning_rate=2e-5, device='cuda'):
    """
    function is a simple train loop
    """
    model.to(device)

    optimizer = AdamW(model.parameters(), lr=learning_rate)

    model.train()
    for epoch in range(num_epochs):
        total_loss = 0.0

        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
        average_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch+1}/{num_epochs} - Average Loss: {average_loss:.4f}")

    print("Training complete!")

Детали:

> Зачем нужно писать ```optimizer.zero_grad```?

В реализации подсчета градиентов во время backward pass у Pytorch градиенты аккумулируются со всех шагов. Читать детали в исходном коде [torch.Autograd](https://pytorch.org/docs/stable/_modules/torch/autograd.html)

Поэтому на каждой итерации чтобы корректно обновить параметры модели, нужно удалить накопленную в прошлом информацию о градиентах, иначе параметры будут обновляться не так как мы ожидаем и модель  вероятно не будет сходиться. Обновление параметров модели происходит в моменте, когда вызывается ```optimizer.step()```- в этот момент оптимизатор итерируется по всем параметрам модели, для которых ```requires_grad=True``` и обновляет соответствующие веса в соответствии с градиентами, которые были посчитаны во время вызова ```loss.backward()```.

У Pytorch есть [хороший туторил](https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html) с введением в Autograd, который описывает как в библиотеке работает автоматическое дифференцирование.

In [ ]:
train(model, train_dataloader)

Epoch 1/2 - Average Loss: 0.3355
Epoch 2/2 - Average Loss: 0.1538
Training complete!


### 4.3 Оценка качества обученной модели

In [ ]:
val_dataset = ExampleDataset(
    texts=imdb_sample["text"].tolist(),
    labels=imdb_sample["label"].tolist(),
    tokenizer=tokenizer,
    max_length=512
    )

val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [ ]:
valid_labels, valid_preds = evaluate(model, val_dataloader)

f1_score(valid_labels, valid_preds)

0.9149130832570905

Модель, которая была обучена на нескольких тысячах примерах показывает почти такое же качество как и модель, которая намного больше. Если обучить модель подольше и потюнить гипепараметры, можно добиться еще бОльшего качества.



In [ ]:
# на случай если нужно освободить GPU память.

model.cpu()
torch.cuda.empty_cache()

 ### Какие проблемы могут возникнуть во время fine tuning?

 - catastrophic foregetting. Если слишком агрессивно дообучать модель, то при обновлении весов модель может слишком сильно переобучиться под задачу и растерять свои базовые знания. Что можно сделать:
* дообучать на все веса, а только часть весов (например, только линейный слой классификатора поверх основной архитектуры)
* использовать регуляризацию весов - например [weight decay](https://medium.com/analytics-vidhya/deep-learning-basics-weight-decay-3c68eb4344e9)
* использовать методы вроде [PEFT](https://github.com/huggingface/peft), которые добавляют отдельные веса под конкретную задачу и обновляют только их, не трогая основную архитектуру. Про PEFT поговорим позже.


## Обратная связь
https://forms.gle/AawczMMJ4dvXvPNk8
